In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
from PIL import Image
import os
import numpy as np

# Configuration
DATA_DIR = "/kaggle/input/tb-data/tbdata/Training_Dataset_TB"
TEST_DIR = "/kaggle/input/tb-data/tbdata/Test_Dataset_TB"
CSV_PATH = "/kaggle/input/tb-data/tbdata/TB_train.csv"
BATCH_SIZE = 32
IMAGE_SIZE = 224
EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load and prepare data
df = pd.read_csv(CSV_PATH)
df = df.rename(columns={'Target': 'label'})  # Keep ID column as is

# Split data
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)

# Calculate class weights
class_counts = df['label'].value_counts()
class_weights = class_counts[0] / class_counts[1]
print(f"Class weights: {class_weights:.2f}")

# Image transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset class
class TBXRayDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['ID'])
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.df.iloc[idx]['label'], dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Create datasets and dataloaders
train_dataset = TBXRayDataset(train_df, DATA_DIR, train_transform)
val_dataset = TBXRayDataset(val_df, DATA_DIR, val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Model setup
weights = models.EfficientNet_B3_Weights.IMAGENET1K_V1
model = models.efficientnet_b3(weights=weights)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
model = model.to(DEVICE)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([class_weights]).to(DEVICE))
optimizer = optim.Adam(model.parameters(), lr=3e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5)

# Training loop
best_f1 = 0
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
    # Validation
    model.eval()
    val_preds = []
    val_true = []
    val_loss = 0.0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images).squeeze()
            
            val_loss += criterion(outputs, labels).item()
            preds = torch.sigmoid(outputs).cpu().numpy()
            
            val_preds.extend(preds)
            val_true.extend(labels.cpu().numpy())
    
    # Calculate metrics
    val_preds_bin = np.array(val_preds) > 0.5
    f1 = f1_score(val_true, val_preds_bin)
    acc = accuracy_score(val_true, val_preds_bin)
    prec = precision_score(val_true, val_preds_bin)
    rec = recall_score(val_true, val_preds_bin)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader)
    
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f}")
    print(f"F1: {f1:.4f} | Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f}")
    
    # Save best model
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), "best_model2.pth")
        print("Saved best model!")

# Generate predictions
model.load_state_dict(torch.load("best_model2.pth"))
model.eval()

test_files = [f for f in os.listdir(TEST_DIR) if f.endswith('.png')]
test_preds = []

with torch.no_grad():
    for file in test_files:
        img_path = os.path.join(TEST_DIR, file)
        image = Image.open(img_path).convert('RGB')
        image = val_test_transform(image).unsqueeze(0).to(DEVICE)
        
        output = torch.sigmoid(model(image)).item()
        test_preds.append({
            'ID': file,
            'Target': 1 if output > 0.5 else 0
        })

# Create submission file
submission_df = pd.DataFrame(test_preds)
submission_df.to_csv('tb_predictions2.csv', index=False)
print("Submission file created: tb_predictions2.csv")